In [1]:
import sympy as sym
import numpy as np
import time
import ae353_spacecraft
import matplotlib.pyplot as plt
import importlib
importlib.reload(ae353_spacecraft)
from sympy.physics import mechanics
mechanics.init_vprinting()
from scipy import linalg
from scipy import signal

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [8]:
#define controller EOM

# Mass and MOI of base
mb = 6.
Jxb = 10.
Jyb = 10.
Jzb = 16.

# Mass and MOI of each wheel
mw = 1.
Jxw = 0.075
Jyw = 0.075
Jzw = 0.125
lw = 1.1

# Define yaw, pitch, roll yaw angles
psi, theta, phi = sym.symbols('psi, theta, phi')

# Define angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# Define torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# Compute resultant torques
lt = sym.nsimplify(lw) * sym.sqrt(2) / 2
T1 = - tau_1 * sym.Matrix([[lt], [0], [lt]])
T2 = - tau_2 * sym.Matrix([[-lt], [0], [lt]])
T3 = - tau_3 * sym.Matrix([[0], [lt], [lt]])
T4 = - tau_4 * sym.Matrix([[0], [-lt], [lt]])
T = T1 + T2 + T3 + T4

# Define MOI of spacecraft and wheels together
Jx = sym.nsimplify(Jxb + 4 * mw * lw**2)
Jy = sym.nsimplify(Jyb + 4 * mw * lw**2)
Jz = sym.nsimplify(Jzb + 4 * mw * lw**2)

# Define rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])

# Define the transformation from angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# Define euler's equations
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# Define equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

f

⎡         w_y⋅sin(φ) + w_z⋅cos(φ)          ⎤
⎢         ───────────────────────          ⎥
⎢                  cos(θ)                  ⎥
⎢                                          ⎥
⎢         w_y⋅cos(φ) - w_z⋅sin(φ)          ⎥
⎢                                          ⎥
⎢wₓ + w_y⋅sin(φ)⋅tan(θ) + w_z⋅cos(φ)⋅tan(θ)⎥
⎢                                          ⎥
⎢     55⋅√2⋅τ₁   55⋅√2⋅τ₂   150⋅w_y⋅w_z    ⎥
⎢   - ──────── + ──────── - ───────────    ⎥
⎢       1484       1484         371        ⎥
⎢                                          ⎥
⎢      55⋅√2⋅τ₃   55⋅√2⋅τ₄   150⋅wₓ⋅w_z    ⎥
⎢    - ──────── + ──────── + ──────────    ⎥
⎢        1484       1484        371        ⎥
⎢                                          ⎥
⎢       -55⋅√2⋅(τ₁ + τ₂ + τ₃ + τ₄)         ⎥
⎢       ───────────────────────────        ⎥
⎣                   2084                   ⎦

In [ ]:
#Define obs eom



In [9]:
# equilibrium point test
f_num = sym.lambdify([psi, theta, phi, w_x, w_y, w_z, tau_1, tau_2, tau_3, tau_4], f)
psie = 0.
thetae = 0.
phie = 1.0
w_xe = 0.
w_ye = 0.
w_ze = 0.
tau_1e = 0.
tau_2e = 0.
tau_3e = 0.
tau_4e = 0.
   
print(f_num(psie, thetae, phie, w_xe, w_ye, w_ze, tau_1e, tau_2e, tau_3e, tau_4e))


#Find A
A_num = sym.lambdify([psi, theta, phi, w_x, w_y, w_z, tau_1, tau_2, tau_3, tau_4], f.jacobian([psi, theta, phi, w_x, w_y, w_z]))
A = A_num(psie, thetae, phie, w_xe, w_ye, w_ze, tau_1e, tau_2e, tau_3e, tau_4e).astype(float)

#Find B
B_num = sym.lambdify([psi, theta, phi, w_x, w_y, w_z, tau_1, tau_2, tau_3, tau_4], f.jacobian([tau_1, tau_2, tau_3, tau_4]))
B = B_num(psie, thetae, phie, w_xe, w_ye, w_ze, tau_1e, tau_2e, tau_3e, tau_4e).astype(float)

print(A)
print(B)

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-0.]]
[[ 0.     0.     0.     0.     0.841  0.54 ]
 [ 0.     0.    -0.     0.     0.54  -0.841]
 [ 0.     0.     0.     1.     0.     0.   ]
 [ 0.     0.     0.     0.    -0.    -0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.   ]]
[[ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]
 [-0.052  0.052  0.     0.   ]
 [ 0.     0.    -0.052  0.052]
 [-0.037 -0.037 -0.037 -0.037]]


Using LQR to find an optimal gain matrix, K:
Define weight matrices Q_c, R_c

In [1]:
# weight matrix Qc (error)
Q_c = np.diag([1000., 1000., 1000., 1., 1., 1.])
print(Q_c)
# weight matrix Rc (effort)
R_c = np.eye(B.shape[1])
print(R_c)

# cost matrix P_c
P_c = linalg.solve_continuous_are(A, B, Q_c, R_c)

# controller gain matrix K
K = linalg.inv(R_c) @ B.T @ P_c


print('Your optimal gain matrix is:\n')
print(K1)

# check eigenvalues to ensure asymptotic stabilty
F = A - B@K1
s = sla.eigvals(F)
print('\n The eigenvalues of your system are:\n')
print(s)

if (s.real < 0).all() == True:
    print('\n The eigenvalues of your system are all of negative real part, \
    \n therefore the closed-loop system is indeed asymptotically stable')
else:
    print('\n The eigenvalues of your system are NOT all of negative real part, please refine your Q and R weights')


NameError: name 'np' is not defined

In [11]:
#check asymptotic stability
# get eigenvalues of ABK
eig_c = linalg.eigvals(A-B@K)
# display the real eigenvalues of ABK
print(eig_c.real)

# display a fancy statement of asymptotic stability
print('Asymptotically stable: ' + str((eig_c.real < 0).all()))

[-1.087 -1.087 -1.083 -1.083 -1.083 -1.083]
Asymptotically stable: True


In [ ]:
alpha, delta = sym.symbols('alpha, delta')
# Scope radius
r = 0.8 / 2.1
# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                              [sym.sin(delta)]])

# Orientation of body frame in space frame
R_body_in_space = Rz * Ry * Rx

# Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

# Position of star in image frame
p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                       [p_star_in_body[2] / p_star_in_body[0]]])

# Sensor model for each star
g = sym.simplify(p_star_in_image, full=True)

for (i, star) in enumerate(simulator.stars):
    #print(f'star {i} : alpha = {star["alpha"]:5.2f} : delta = {star["delta"]:5.2f}')
    alphae = star["alpha"]
    deltae = star["delta"]

    g.subs([(psi, 0), (theta, 0), (phi, 0), (alpha, alphae), (delta, deltae)]).tolist()

    j_num = sym.lambdify((psi, theta, phi, w_x, w_y, w_z), g.subs([(alpha, alphae), (delta, deltae)]).jacobian([psi, theta, phi, w_x, w_y, w_z]))
    C = j_num(0, 0, 0, 0, 0, 0).astype(float)
    print(C.tolist())

In [12]:
#LQR for obs (L)

# weight matrix Qo ﴾sensor noise﴿
Q_o = np.diag([1., 1., 1., 1., 1., 1.])
print(Q_o)
# weight matrix Ro ﴾process noise﴿
R_o = np.diag([1., 1., 1., 1., 1., 1.])
print(R_o)

# cost matrix P_c
P_o = linalg.solve_continuous_are(A.T, C.T, linalg.inv(R_o), linalg.inv(Q_o))

# observer injection matrix L
L = P_o @ C.T @ Q_o
print('L (observer injection matrix)')
print(L)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


NameError: name 'C' is not defined

In [13]:
#check asym stable
# get eigenvalues of ALC
eig_o = linalg.eigvals(A-L@C)
# display the real eigenvalues of ABK
print(eig_o.real)
# display a fancy statement of asymptotic stability
print('Asymptotically stable: ' + str((eig_o.real < 0).all()))

NameError: name 'L' is not defined